* 在数加系统内获得全部 features 的值
* 复制 SQL 里面的 feature 到 csv 文件中
* 读取 csv 文件到 pandas
* 获得 feature 的全部列表
* 构建 sql 语言函数
* 生成 sql 语言

In [1]:
import numpy as np 
import pandas as pd

In [33]:
(0.89 + 0.94 + 0.79)/3

0.9366666666666666

In [ ]:
,brand_id, car_height, car_length, car_width, compartment
, cylinder_number, department_id, displacement, driven_type_id
, emission_standards_id, equipment_quality, front_track
, fuel_type_id, gearbox_type, if_charging, if_luxurious_id
, if_mpv_id, level_id, newenergy_type_id, power
, price_level, rated_passenger, rear_track, total_quality
, tr, type_id, wheelbase

In [9]:
def get_sql(feature):
    """
    1. 读取各个特征的 csv 文件进入 pandas
    2. 得到全部值的列表
    3. 打印全部值的 sql 语言 
    """
    # 读取 pandas 文件
    df = pd.read_csv('../../raw/LiChuan/features/{}.csv'.format(feature))
    
    # 得到全部 brand_id 的列表
    feature_ids = []
    for index, row in df.iterrows():
        feature_ids.append(str(row.values[0]))
        
        
    """
    用于 One-hot 编码的 sql 语言生成
    """
    for i in feature_ids:
        if ";" in i:
            x = i.split(";")[0] + "_" + i.split(";")[1] # 将 xx;xx, 形式转换成 xx_xx 形式, 这样 sql 才能读取
            print('case when {}={} then 1.0 else 0.0 end as {}_{},'.format(feature, i, feature, str(x)))
        else:
            print('case when {}={} then 1.0 else 0.0 end as {}_{},'.format(feature, i, feature, str(i)))
    
    """
    用于相同 class_id 进行 merge 的 sql 语言生成
    """
    for j in feature_ids:
        if ";" in j:
            m = feature + "_" + j.split(";")[0] + "_" + j.split(";")[1] 
            print(",case when sum(" + m + ")!= 0 then 1 else 0 end as " + m)
        else:
            n = feature + "_" + j
            print(",case when sum(" + n + ")!= 0 then 1 else 0 end as " + n)
            
    """
    用于特征序列的 sql 语言生成
    """
    for j in feature_ids:
        if ";" in j:
            m = feature + "_" + j.split(";")[0] + "_" + j.split(";")[1] # 将 xx;xx, 形式转换成 xx_xx 形式, 这样 sql 才能读取
            print("," + m)
        else:
            n = feature + "_" + j
            print("," + n)

In [ ]:
# 需要进行 one-hot 编码的特征名
features = ['brand_id', 'compartment', 'type_id', 'level_id', 'department_id' ,'tr', 'gearbox_type'
            , 'if_charging', 'driven_type_id', 'fuel_type_id'
            ,'newenergy_type_id', 'emission_standards_id' , 'if_mpv_id(不用编码)', 'if_luxurious_id(不用编码)'
            ,'cylinder_number', ]

In [13]:
# 需要进行 one-hot 编码的特征名
features = ['brand_id', 'compartment', 'type_id', 'level_id', 'department_id' ,'tr', 'gearbox_type'
            , 'if_charging', 'driven_type_id', 'fuel_type_id'
            ,'newenergy_type_id', 'emission_standards_id' ,'cylinder_number', ]

In [14]:
# feautre
for feature in features:
    get_sql(feature)

,case when sum(brand_id_15)!= 0 then 1 else 0 end as brand_id_15
,case when sum(brand_id_18)!= 0 then 1 else 0 end as brand_id_18
,case when sum(brand_id_21)!= 0 then 1 else 0 end as brand_id_21
,case when sum(brand_id_27)!= 0 then 1 else 0 end as brand_id_27
,case when sum(brand_id_30)!= 0 then 1 else 0 end as brand_id_30
,case when sum(brand_id_36)!= 0 then 1 else 0 end as brand_id_36
,case when sum(brand_id_39)!= 0 then 1 else 0 end as brand_id_39
,case when sum(brand_id_42)!= 0 then 1 else 0 end as brand_id_42
,case when sum(brand_id_48)!= 0 then 1 else 0 end as brand_id_48
,case when sum(brand_id_57)!= 0 then 1 else 0 end as brand_id_57
,case when sum(brand_id_60)!= 0 then 1 else 0 end as brand_id_60
,case when sum(brand_id_63)!= 0 then 1 else 0 end as brand_id_63
,case when sum(brand_id_66)!= 0 then 1 else 0 end as brand_id_66
,case when sum(brand_id_69)!= 0 then 1 else 0 end as brand_id_69
,case when sum(brand_id_84)!= 0 then 1 else 0 end as brand_id_84
,case when sum(brand_id_9

In [31]:
# 2012-2017 年
for i in range(1, 13):
    if i in range(10):
        print (
        """
        DROP TABLE IF EXISTS stuff_{};
        CREATE TABLE stuff_{}
        AS
        SELECT a.sale_date, a.city_id, a.class_id, a.sale_quantity, b.how_many_month_sale
        FROM (
            SELECT sale_date, city_id, class_id, sale_quantity
            FROM stuff_lichuan0306_feature_11_how_many_month_sale a
            WHERE sale_date = '20170{}'
        ) a
        LEFT OUTER JOIN (
            SELECT city_id, class_id, how_many_month_sale
            FROM stuff_lichuan0306_feature_11_how_many_month_sale
            WHERE sale_date = '20170{}'
        ) b
        ON a.city_id = b.city_id
            AND a.class_id = b.class_id;

        INSERT OVERWRITE TABLE 
                stuff_{}
        Select sale_date, city_id, class_id, sale_quantity
            ,case
                when sale_quantity != 0
                    then how_many_month_sale + 1
                else how_many_month_sale
            end as how_many_month_sale
        from stuff_{};

        INSERT OVERWRITE TABLE stuff_lichuan0306_feature_11_how_many_month_sale
        SELECT a.sale_date, a.city_id, a.class_id, a.sale_quantity
            , CASE 
                WHEN a.sale_date = '20170{}' THEN b.how_many_month_sale
                ELSE a.how_many_month_sale
            END AS how_many_month_sale
        FROM stuff_lichuan0306_feature_11_how_many_month_sale a
        LEFT OUTER JOIN stuff_{} b
        ON a.sale_date = b.sale_date
            AND a.city_id = b.city_id
            AND a.class_id = b.class_id;
        """
        .format(i, i, i, i-1, i, i, i, i))
    else:
        print (
        """
        DROP TABLE IF EXISTS stuff_{};
        CREATE TABLE stuff_{}
        AS
        SELECT a.sale_date, a.city_id, a.class_id, a.sale_quantity, b.how_many_month_sale
        FROM (
            SELECT sale_date, city_id, class_id, sale_quantity
            FROM stuff_lichuan0306_feature_11_how_many_month_sale a
            WHERE sale_date = '2017{}'
        ) a
        LEFT OUTER JOIN (
            SELECT city_id, class_id, how_many_month_sale
            FROM stuff_lichuan0306_feature_11_how_many_month_sale
            WHERE sale_date = '2017{}'
        ) b
        ON a.city_id = b.city_id
            AND a.class_id = b.class_id;

        INSERT OVERWRITE TABLE 
                stuff_{}
        Select sale_date, city_id, class_id, sale_quantity
            ,case
                when sale_quantity != 0
                    then how_many_month_sale + 1
                else how_many_month_sale
            end as how_many_month_sale
        from stuff_{};

        INSERT OVERWRITE TABLE stuff_lichuan0306_feature_11_how_many_month_sale
        SELECT a.sale_date, a.city_id, a.class_id, a.sale_quantity
            , CASE 
                WHEN a.sale_date = '2017{}' THEN b.how_many_month_sale
                ELSE a.how_many_month_sale
            END AS how_many_month_sale
        FROM stuff_lichuan0306_feature_11_how_many_month_sale a
        LEFT OUTER JOIN stuff_{} b
        ON a.sale_date = b.sale_date
            AND a.city_id = b.city_id
            AND a.class_id = b.class_id;
        """
        .format(i, i, i, i-1, i, i, i, i))


        DROP TABLE IF EXISTS stuff_1;
        CREATE TABLE stuff_1
        AS
        SELECT a.sale_date, a.city_id, a.class_id, a.sale_quantity, b.how_many_month_sale
        FROM (
            SELECT sale_date, city_id, class_id, sale_quantity
            FROM stuff_lichuan0306_feature_11_how_many_month_sale a
            WHERE sale_date = '201701'
        ) a
        LEFT OUTER JOIN (
            SELECT city_id, class_id, how_many_month_sale
            FROM stuff_lichuan0306_feature_11_how_many_month_sale
            WHERE sale_date = '201700'
        ) b
        ON a.city_id = b.city_id
            AND a.class_id = b.class_id;

        INSERT OVERWRITE TABLE 
                stuff_1
        Select sale_date, city_id, class_id, sale_quantity
            ,case
                when sale_quantity != 0
                    then how_many_month_sale + 1
                else how_many_month_sale
            end as how_many_month_sale
        from stuff_1;

        INSERT OVERWRITE TABLE st